In [ ]:
import rishi_utils as ru
from matplotlib

In [ ]:
PATH_TO_DATA_FILE = '/data/rgur/efrc/paper_data/isotherm_models/geo/results_iso_code_1011_fold_2_10_47AM_on_May_08_2020.csv'

In [ ]:
df = ru.pd_load(PATH_TO_DATA_FILE)

test = df[df['Class'] == 'Test']
true = test['Truth'].tolist()
pred = test['Prediction'].tolist()
pred = [max(i, 0) for i in pred]

%matplotlib inline
plt.scatter(true, pred)